In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from scipy.signal import find_peaks
# import chart_studio.plotly as py
# import cufflinks as cf
%matplotlib inline

# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode(connected = True)
# cf.go_offline()
# import plotly.graph_objects as go
import plotly.graph_objects as go
import plotly.express as px
pd.options.plotting.backend = "plotly"

In [13]:
def rename_columns_partially(df, old_part, new_part):
  # df = df.rename(column=new_column_names)
  new_column_names = {col: col.replace(old_part, new_part) for col in df.columns}
  df = df.rename(columns=new_column_names)
  return df


def data_clean(df, file_path, i):
  DF_FILE = df[10:]
  DF_FILE.columns = df.iloc[9]
  DF_FILE = DF_FILE.reset_index(drop=True)
  if (i == 0):
    DF_FILE = DF_FILE.drop(['Frequency','Port ID','Port Name','Step Axis'], axis = 1)
  else:
    DF_FILE = DF_FILE.drop(['Frequency','Port ID','Port Name', 'Step Axis', 'Pol. Axis', 'Scan Axis'], axis = 1)
  DF_FILE = DF_FILE.astype('float64')
  colu = DF_FILE.columns[2]
  filename = file_path.split('/')[-1]
  filename = filename[:-4]
  DF_FILE = rename_columns_partially(DF_FILE, colu[0:10], filename)
  return DF_FILE


def import_multiple_csv_files(number_of_files):

  dataframes = []

  for i in range(number_of_files):
    file_path = input(f"Enter the file path for file {i+1}: ")
    df = pd.read_csv(file_path)
    df = data_clean(df, file_path, i)
    dataframes.append(df)

  return dataframes

number_of_files = int(input("Enter the number of files: "))
data = import_multiple_csv_files(number_of_files)

df = pd.DataFrame(data[0])
i=1
while i<number_of_files:
  df = pd.concat([df, data[i]], axis=1)
  i+=1

Enter the number of files: 8
Enter the file path for file 1: /content/drive/MyDrive/Canopy 2x2 SM Rad Pattern/HP-450/Azimuth Cut.csv
Enter the file path for file 2: /content/drive/MyDrive/Canopy 2x2 SM Rad Pattern/HP-450/Elevation Cut.csv
Enter the file path for file 3: /content/drive/MyDrive/Canopy 2x2 SM Rad Pattern/HP-650/Azimuth Cut.csv
Enter the file path for file 4: /content/drive/MyDrive/Canopy 2x2 SM Rad Pattern/HP-650/Elevation Cut.csv
Enter the file path for file 5: /content/drive/MyDrive/Canopy 2x2 SM Rad Pattern/VP-450/Azimuth Cut.csv
Enter the file path for file 6: /content/drive/MyDrive/Canopy 2x2 SM Rad Pattern/VP-450/Elevation Cut.csv
Enter the file path for file 7: /content/drive/MyDrive/Canopy 2x2 SM Rad Pattern/VP-650/Azimuth Cut.csv
Enter the file path for file 8: /content/drive/MyDrive/Canopy 2x2 SM Rad Pattern/VP-650/Elevation Cut.csv


In [14]:
df

9     Pol. Axis  Scan Axis  Azimuth Cut @ 5.100 GHz  Azimuth Cut @ 5.125 GHz  \
0           0.0    -180.00                  -12.195                  -12.468   
1           0.0    -179.75                  -12.217                  -12.579   
2           0.0    -179.50                  -12.388                  -12.803   
3           0.0    -179.25                  -12.646                  -13.133   
4           0.0    -179.00                  -13.022                  -13.554   
...         ...        ...                      ...                      ...   
2875       90.0     178.75                  -17.992                  -17.312   
2876       90.0     179.00                  -18.123                  -17.636   
2877       90.0     179.25                  -18.364                  -17.924   
2878       90.0     179.50                  -18.633                  -18.299   
2879       90.0     179.75                  -18.942                  -18.657   

9     Azimuth Cut @ 5.150 GHz  Azimuth Cut @ 5.175 GHz  \
0                     -14.152                  -17.979   
1                     -14.280                  -18.153   
2                     -14.561                  -18.452   
3                     -14.922                  -18.909   
4                     -15.453                  -19.550   
...                       ...                      ...   
2875                  -17.893                  -19.564   
2876                  -18.227                  -19.943   
2877                  -18.674                  -20.472   
2878                  -19.187                  -21.030   
2879                  -19.712                  -21.673   

9     Azimuth Cut @ 5.200 GHz  Azimuth Cut @ 5.225 GHz  \
0                     -19.912                  -22.342   
1                     -19.961                  -22.432   
2                     -20.241                  -22.712   
3                     -20.628                  -23.085   
4                     -21.249                  -23.568   
...                       ...                      ...   
2875                  -22.179                  -24.277   
2876                  -22.770                  -25.098   
2877                  -23.449                  -25.996   
2878                  -24.174                  -27.029   
2879                  -24.944                  -28.181   

9     Azimuth Cut @ 5.250 GHz  Azimuth Cut @ 5.275 GHz  ...  \
0                     -25.302                  -24.925  ...   
1                     -24.813                  -23.546  ...   
2                     -24.409                  -22.464  ...   
3                     -24.043                  -21.601  ...   
4                     -23.689                  -20.899  ...   
...                       ...                      ...  ...   
2875                  -26.183                  -29.153  ...   
2876                  -26.986                  -29.879  ...   
2877                  -27.960                  -30.811  ...   
2878                  -29.023                  -31.831  ...   
2879                  -30.179                  -33.123  ...   

9     Elevation Cut @ 6.975 GHz  Elevation Cut @ 7 GHz  \
0                       -17.170                -18.323   
1                       -16.464                -17.534   
2                       -15.893                -16.846   
3                       -15.348                -16.265   
4                       -14.820                -15.750   
...                         ...                    ...   
2875                    -18.408                -18.557   
2876                    -18.857                -18.753   
2877                    -19.402                -19.097   
2878                    -19.976                -19.489   
2879                    -20.539                -19.990   

9     Elevation Cut @ 7.025 GHz  Elevation Cut @ 7.050 GHz  \
0                       -19.543                    -21.294   
1                       -18.486                    -20.051   
2    

In [22]:
df.dtypes

9
Pol. Axis                    float64
Scan Axis                    float64
Azimuth Cut @ 5.100 GHz      float64
Azimuth Cut @ 5.125 GHz      float64
Azimuth Cut @ 5.150 GHz      float64
                              ...   
Elevation Cut @ 7.100 GHz    float64
Elevation Cut @ 7.125 GHz    float64
Elevation Cut @ 7.150 GHz    float64
Elevation Cut @ 7.175 GHz    float64
Elevation Cut @ 7.200 GHz    float64
Length: 682, dtype: object

In [ ]:
df.shape

(2880, 682)

In [15]:
rows = int(df.shape[0]/2)
dfp = df.iloc[:rows,:]

In [16]:
dfp

9     Pol. Axis  Scan Axis  Azimuth Cut @ 5.100 GHz  Azimuth Cut @ 5.125 GHz  \
0           0.0    -180.00                  -12.195                  -12.468   
1           0.0    -179.75                  -12.217                  -12.579   
2           0.0    -179.50                  -12.388                  -12.803   
3           0.0    -179.25                  -12.646                  -13.133   
4           0.0    -179.00                  -13.022                  -13.554   
...         ...        ...                      ...                      ...   
1435        0.0     178.75                  -13.313                  -13.329   
1436        0.0     179.00                  -12.903                  -12.975   
1437        0.0     179.25                  -12.587                  -12.703   
1438        0.0     179.50                  -12.342                  -12.518   
1439        0.0     179.75                  -12.223                  -12.452   

9     Azimuth Cut @ 5.150 GHz  Azimuth Cut @ 5.175 GHz  \
0                     -14.152                  -17.979   
1                     -14.280                  -18.153   
2                     -14.561                  -18.452   
3                     -14.922                  -18.909   
4                     -15.453                  -19.550   
...                       ...                      ...   
1435                  -14.830                  -18.612   
1436                  -14.513                  -18.384   
1437                  -14.310                  -18.101   
1438                  -14.151                  -17.983   
1439                  -14.094                  -17.952   

9     Azimuth Cut @ 5.200 GHz  Azimuth Cut @ 5.225 GHz  \
0                     -19.912                  -22.342   
1                     -19.961                  -22.432   
2                     -20.241                  -22.712   
3                     -20.628                  -23.085   
4                     -21.249                  -23.568   
...                       ...                      ...   
1435                  -20.985                  -23.405   
1436                  -20.620                  -23.017   
1437                  -20.299                  -22.685   
1438                  -20.111                  -22.449   
1439                  -19.947                  -22.322   

9     Azimuth Cut @ 5.250 GHz  Azimuth Cut @ 5.275 GHz  ...  \
0                     -25.302                  -24.925  ...   
1                     -24.813                  -23.546  ...   
2                     -24.409                  -22.464  ...   
3                     -24.043                  -21.601  ...   
4                     -23.689                  -20.899  ...   
...                       ...                      ...  ...   
1435                  -28.928                  -48.948  ...   
1436                  -28.194                  -36.995  ...   
1437                  -27.340                  -32.041  ...   
1438                  -26.539                  -28.950  ...   
1439                  -25.874                  -26.735  ...   

9     Elevation Cut @ 6.975 GHz  Elevation Cut @ 7 GHz  \
0                       -17.170                -18.323   
1                       -16.464                -17.534   
2                       -15.893                -16.846   
3                       -15.348                -16.265   
4                       -14.820                -15.750   
...                         ...                    ...   
1435                    -21.974                -25.312   
1436                    -21.144                -24.065   
1437                    -20.260                -22.662   
1438                    -19.381                -21.261   
1439                    -18.514                -19.946   

9     Elevation Cut @ 7.025 GHz  Elevation Cut @ 7.050 GHz  \
0                       -19.543                    -21.294   
1                       -18.486                    -20.051   
2    

In [ ]:
rows = int(df.shape[0]/2)
dfp = df.iloc[:rows,:]

fig = px.line(df, x = df.iloc[:,1], y = df.columns[2:])

# , template='plotly_dark' markers = True

fig.update_layout(
    width=1900,  # Set the width of the plot
    height=1000,   # Set the height of the plot
    title = 'CrossPol'
)

# fig.update_layout(
#     updatemenus=[
#         dict(active=0,
#             buttons=list([
#             dict(label="None",
#                  method="update",
#                  args=[{"visible":df},
#                        {"title":"ALL"}]),
#             dict(label="No CrossPol",
#                  method="update",
#                  args=[{"visible":df.iloc[:rows,:]},
#                        {"title":"No CrossPol"}]),
#         ]),
#         )
#     ]
# )

fig.show()

In [20]:
# rows = int(df.shape[0]/2)
# dfp = df.iloc[:rows,:]

fig = px.line(dfp, x = dfp.iloc[:,1], y = dfp.columns[2:])
# , template='plotly_dark'

fig.update_layout(
    width=1900,  # Set the width of the plot
    height=1000,   # Set the height of the plot
    title = 'n'
)



fig.show()

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [23]:
rows = int(df.shape[0]/2)
dfp = df.iloc[:rows,:]
# file_selector=

fig = px.line(dfp.iloc[:,], x = dfp.iloc[:,1], y = dfp.columns[2:])
# , template='plotly_dark'

fig.update_layout(
    width=1900,  # Set the width of the plot
    height=1000,   # Set the height of the plot
    title = 'n'
)



fig.show()

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().